In [1]:
%run utils/devtools.py

In [2]:
%reloadmypkg utils

import pandas as pd
from utils.url2platform import *
from utils.standardise_url import *
from utils.fetch_info import *
from utils.dump_df_to_json import *

✅ Reloaded package 'utils' and its submodules.


In [3]:
sheet_name = "../raw-data/neetcode/75-sheet.json"
export_file = "../cleaned-data/neetcode/75.json"

In [4]:
with open(sheet_name) as file:
    raw = json.load(file)

df = pd.DataFrame(raw, columns=['url'])

In [5]:
df.head(2)

,url
0,https://leetcode.com/problems/contains-duplicate/
1,https://leetcode.com/problems/valid-anagram/


In [6]:
df.shape

(75, 1)

In [7]:
# Find the duplicates

dupes_df = df[df.duplicated(subset="url", keep=False)]
dupes_df.shape

(0, 1)

In [8]:
import nest_asyncio
nest_asyncio.apply()

standardised_df = await standardise_urls_async(df, "neetcode-75-urls-expanded.json")

Filter list is empty. Trying to expand all 75 URLs.

Processing batch 1/1
Loaded 0 results from checkpoint
Processing 75 pending URLs in this batch


Expanding URLs: 100%|██████████| 75/75 [00:00<00:00, 82.70it/s]

Progress saved to neetcode-75-urls-expanded.json
----------------------------------------------------------------------
Progress saved to neetcode-75-urls-expanded.json
----------------------------------------------------------------------

75/75 URLs expanded in 2.11 seconds.


In [9]:
standardised_df.shape

(75, 3)

In [10]:
# Adding type column
standardised_df['type'] = standardised_df['expanded_stripped_url'].apply(
    lambda url: "problem" if is_problem_url(url) else "tutorial"
)

In [11]:
# Adding id_base, id, title, platform columns
import nest_asyncio
nest_asyncio.apply()

final_df = await fetch_info(standardised_df, "neetcode-75-info-added.json", resume=True)

Found 75 URLs.

Batch 1/1: items 0-74
Loaded 0 from checkpoint
Processing 75 URLs...


Fetching items: 100%|██████████| 75/75 [00:02<00:00, 29.97it/s]

Saved progress (75 items) to neetcode-75-info-added.json
------------------------------
Saved progress (75 items) to neetcode-75-info-added.json
------------------------------

Fetched 75/75 items in 2.5s.


In [12]:
final_df.shape

(75, 8)

In [13]:
dump_df_to_json(final_df, export_file)